# Analysis Scripts

Schema:

Titolo

Data -> Json

Mnist 

    Analisi all'aumentare del numero di nodi

        Real-Time

        Persistent Data

    Analisi all'aumentare del numero di round (solo real-time)

    Analisi al diminuire del numero di nodi scelti ad ogni round

Fashion MNIST

    Analisi all'aumentare del numero di nodi

        Real-Time

        Persistent Data

    Analisi all'aumentare del numero di round (solo real-time)

    Analisi al diminuire del numero di nodi scelti ad ogni round

## CodeBase

In [ ]:
def parse_percentage(value):
    """Convert percentage string like '80%' to float."""
    return int(value.strip('%'))

def convert_to_json(data_str):
    """Convert multiline tabular string to list of dicts in desired JSON format."""
    json_list = []
    lines = data_str.strip().splitlines()
    
    for line in lines:
        # Skip empty lines
        if not line.strip():
            continue

        parts = line.split()
        
        if len(parts) < 5:
            continue  # Incomplete row, skip
        
        total_clients = int(parts[0])
        rounds = int(parts[1])
        clients_per_round_percent = parse_percentage(parts[2])
        accuracy = float(parts[3])
        byzantine_clients = int(parts[4])

        json_list.append({
            "total_clients": total_clients,
            "rounds": rounds,
            "clients_per_round_percent": clients_per_round_percent,
            "byzantine_clients": byzantine_clients,
            "accuracy": accuracy
        })
    
    return json_list

In [ ]:
def get_by_client_selection(client_selection, ds, num_clients, rounds) -> list[list[dict[str, any]]]:
    cs = \
    [
        [
            {
                "Accuracy": x["accuracy"], 
                "Byzantine": x["byzantine_clients"],
                "Total Clients": x["total_clients"],
                "Rounds": x["rounds"]
            }
            for x in ds
            if x["total_clients"] == total and 
            x["rounds"] == rnd and 
            x["clients_per_round_percent"] == client_selection
        ]
        for total in num_clients
        for rnd in rounds
    ]

    cs.sort(key=lambda x : x[0]['Byzantine'])

    return cs

def get_seq(data: list[list[dict[str, any]]]) -> list[str]:
    return [
        "\n" + str(group[0]['Total Clients']) + " clients, " + str(group[0]['Rounds']) + " rounds: " +
        ' -> '.join([str(x['Accuracy']) for x in group])
        for group in data
    ]


def get_performance_decrease(client_selection, ds, num_clients, rounds):
    cs = get_by_client_selection(client_selection, ds, num_clients, rounds)
    seq = get_seq(cs)
    return seq

In [ ]:
from numpy import mean

def mean_at_byzantine_n(n, n_round, client_selection, ds):
    l = []

    for x in ds:
        if x['rounds'] == n_round and x['clients_per_round_percent'] == client_selection and x['byzantine_clients'] == n-1:
            for y in ds:
                if y['rounds'] == n_round and y['clients_per_round_percent'] == client_selection and y['byzantine_clients'] == n and x['total_clients'] == y['total_clients']:
                    l.append(x['accuracy']-y['accuracy'])
    return mean(l)                

## Convert Data into JSON

In [ ]:
mnist_data_rtime = """
1             1                     100%                     0.9779               0
1             1                     100%                     0.7674               1
2             1                     100%                     0.9672               0
2             1                     100%                     0.7678               1
3             1                     100%                     0.9588               0
4             1                     100%                     0.9511               0
4             1                     80%                      0.9505               0
4             1                     70%                      0.9510               0
5             1                     100%                     0.9474               0
5             1                     80%                      0.9463               0
5             1                     70%                      0.9479               0
6             1                     100%                     0.9425               0
6             1                     80%                      0.9405               0
6             1                     70%                      0.9421               0
7             1                     100%                     0.9396               0
8             1                     100%                     0.9377               0
9             1                     100%                     0.9327               0
1             1                     100%                     0.9291               0
2             2                     100%                     0.9688               0
3             2                     100%                     0.9621               0
4             2                     100%                     0.9539               0
4             2                     80%                      0.9515               0
4             2                     70%                      0.9527               0
5             2                     100%                     0.9495               0
5             2                     80%                      0.9484               0
5             2                     70%                      0.9493               0
6             2                     100%                     0.9452               0
6             2                     80%                      0.9439               0
6             2                     70%                      0.9427               0
2             3                     100%                     0.9678               0
3             3                     100%                     0.9607               0
4             3                     100%                     0.9537               0
4             1                     100%                     0.9154               1
4             1                     100%                     0.7508               2
4             1                     100%                     0.7492               3
4             1                      80%                     0.8630               1
4             1                      80%                     0.7489               2
4             1                      80%                     0.7466               3
4             1                      70%                     0.7511               1
4             1                      70%                     0.7511               2
4             1                      70%                     0.7511               3
5             1                     100%                     0.9277               1
5             1                     100%                     0.7883               2
5             1                     100%                     0.7462               3
5             1                      80%                     0.9049               1
5             1                      80%                     0.7506               2
5             1                      80%                     0.7506               3
5             1                      70%                     0.9455               1
5             1                      70%                     0.8440               2
5             1                      70%                     0.8572               3
6             1                     100%                     0.9251               1
6             1                     100%                     0.8570               2
6             1                     100%                     0.7419               3
6             1                      80%                     0.8997               1
6             1                      80%                     0.8942               2
6             1                      80%                     0.7410               3
6             1                      70%                     0.9411               1
6             1                      70%                     0.9140               2
6             1                      70%                     0.7395               3
4             2                     100%                     0.9533               1
4             2                     100%                     0.9524               2
4             2                     100%                     0.9523               3
4             2                      80%                     0.9528               1
4             2                      80%                     0.9514               2
4             2                      80%                     0.9521               3
4             2                      70%                     0.9528               1
4             2                      70%                     0.9517               2
4             2                      70%                     0.9517               3
5             2                     100%                     0.9511               1
5             2                     100%                     0.9496               2
5             2                     100%                     0.9504               3
5             2                      80%                     0.9496               1
5             2                      80%                     0.9483               2
5             2                      80%                     0.9481               3
5             2                      70%                     0.9480               1
5             2                      70%                     0.9488               2
5             2                      70%                     0.9474               3
6             2                     100%                     0.9446               1
6             2                     100%                     0.9439               2
6             2                     100%                     0.9433               3
6             2                      80%                     0.9428               1
6             2                      80%                     0.9437               2
6             2                      80%                     0.9420               3
6             2                      70%                     0.9428               1
6             2                      70%                     0.9434               2
6             2                      70%                     0.9432               3
"""

mnist_data = """
5                 5               100%                    0.9758               0
5                 5               80%                     0.9762               0
5                 5               70%                     0.9788               0
7                 5               100%                    0.9733               0
7                 5               80%                     0.9737               0
7                 5               70%                     0.9727               0
10                5               100%                    0.9677               0
10                5               80%                     0.9682               0
10                5               70%                     0.9681               0
5                 5              100%                     0.9736               1
5                 5              100%                     0.9691               2
5                 5              100%                     0.9321               3
5                 5              100%                     0.7674               5
5                 5               80%                     0.9738               1
5                 5               80%                     0.9729               2
5                 5               80%                     0.9579               3
5                 5               80%                     0.7676               5
5                 5               70%                     0.9759               1
5                 5               70%                     0.9750               2
5                 5               70%                     0.9658               3
5                 5               70%                     0.7684               5
7                 5              100%                     0.9718               1
7                 5              100%                     0.9678               2
7                 5              100%                     0.9589               3
7                 5              100%                     0.7964               5
7                 5               80%                     0.9689               1
7                 5               80%                     0.9620               2
7                 5               80%                     0.9608               3
7                 5               80%                     0.7937               5
7                 5               70%                     0.9688               1
7                 5               70%                     0.9700               2
7                 5               70%                     0.9701               3
7                 5               70%                     0.9081               5
10                5              100%                     0.9676               1
10                5              100%                     0.9656               2
10                5              100%                     0.9625               3
10                5              100%                     0.9439               5
10                5               80%                     0.9657               1
10                5               80%                     0.9638               2
10                5               80%                     0.9610               3
10                5               80%                     0.9441               5
10                5               70%                     0.9662               1
10                5               70%                     0.9615               2
10                5               70%                     0.9519               3
10                5               70%                     0.9315               5
"""

fashionmnist_data = """
5          5            100%                0.914               0
5          5            80%                 0.913               0
5          5            70%                 0.9132              0
7          5            100%                0.9117              0
7          5            80%                 0.91                0
7          5            70%                 0.9118              0
10         5            100%                0.9101              0
10         5            80%                 0.9067              0
10         5            70%                 0.9078              0
5          5            100%                0.9104              1
5          5            80%                 0.9092              1
5          5            70%                 0.9084              1
7          5            100%                0.9108              1
7          5            80%                 0.9093              1
7          5            70%                 0.9058              1
10         5            100%                0.9115              1
10         5            80%                 0.9072              1
10         5            70%                 0.9094              1
5          5            100%                0.8976              2
5          5            80%                 0.908               2
5          5            70%                 0.9096              2
7          5            100%                0.9042              2
7          5            80%                 0.9039              2
7          5            70%                 0.8959              2
10         5            100%                0.9062              2
10         5            80%                 0.9045              2
10         5            70%                 0.9028              2
5          5            100%                0.7859              3
5          5            80%                 0.8727              3
5          5            70%                 0.9011              3
7          5            100%                0.8955              3
7          5            80%                 0.9018              3
7          5            70%                 0.9049              3
10         5            100%                0.9063              3
10         5            80%                 0.8932              3
10         5            70%                 0.8986              3
"""

fashionmnist_data_rtime = """
4            1            100%           0.9027         0
4            1            80%            0.9006         0
4            1            70%            0.8908         0
4            2            100%           0.9012         0
4            2            80%            0.8997         0
4            2            70%            0.8960         0
5            1            100%           0.9005         0
5            1            80%            0.8987         0
5            1            70%            0.8968         0
5            2            100%           0.8978         0
5            2            80%            0.8974         0
5            2            70%            0.8938         0
6            1            100%           0.8976         0
6            1            80%            0.8968         0
6            1            70%            0.8953         0
6            2            100%           0.8940         0
6            2            80%            0.8922         0
6            2            70%            0.8938         0
4            1            100%           0.8982         1
4            1            80%            0.8600         1
4            1            70%            0.7664         1
4            2            100%           0.9023         1
4            2            80%            0.8966         1
4            2            70%            0.8975         1
5            1            100%           0.8987         1
5            1            80%            0.8844         1
5            1            70%            0.8978         1
5            2            100%           0.9010         1
5            2            80%            0.8982         1
5            2            70%            0.8972         1
6            1            100%           0.8949         1
6            1            80%            0.8793         1
6            1            70%            0.8790         1
6            2            100%           0.8958         1
6            2            80%            0.8932         1
6            2            70%            0.8981         1
4            1            100%           0.7323         2
4            1            80%            0.7121         2
4            1            70%            0.7071         2
4            2            100%           0.9028         2
4            2            80%            0.8963         2
4            2            70%            0.8982         2
5            1            100%           0.8057         2
5            1            80%            0.7584         2
5            1            70%            0.8621         2
5            2            100%           0.8990         2
5            2            80%            0.9002         2
5            2            70%            0.8962         2
6            1            100%           0.8666         2
6            1            80%            0.8790         2
6            1            70%            0.8913         2
6            2            100%           0.8923         2
6            2            80%            0.8953         2
6            2            70%            0.8940         2
4            1            100%           0.7110         3
4            1            80%            0.7096         3
4            1            70%            0.7090         3
4            2            100%           0.9027         3
4            2            80%            0.8974         3
4            2            70%            0.8948         3
5            1            100%           0.7099         3
5            1            80%            0.7113         3
5            1            70%            0.8298         3
5            2            100%           0.8981         3
5            2            80%            0.8996         3
5            2            70%            0.8957         3
6            1            100%           0.7117         3
6            1            80%            0.7222         3
6            1            70%            0.7069         3
6            2            100%           0.8922         3
6            2            80%            0.8909         3
6            2            70%            0.8881         3
"""


In [ ]:
mnist_data = convert_to_json(mnist_data)
mnist_data_rtime = convert_to_json(mnist_data_rtime)
fashionmnist_data = convert_to_json(fashionmnist_data)
fashionmnist_data_rtime = convert_to_json(fashionmnist_data_rtime)

## MNIST Analysis

### Performance analysis as the total number of nodes increases

#### Real-Time Task

In [ ]:
for x in get_performance_decrease(100, mnist_data_rtime, [4,5,6], [1, 2]):
    print(x)

In [ ]:
for x in get_performance_decrease(80, mnist_data_rtime, [4,5,6], [1, 2]):
    print(x)

In [ ]:
for x in get_performance_decrease(70, mnist_data_rtime, [4,5,6], [1, 2]):
    print(x)

#### Persistent Datasets Task

In [ ]:
for x in get_performance_decrease(100, mnist_data, [5,7,10], [5]):
    print(x)

In [ ]:
for x in get_performance_decrease(80, mnist_data, [5,7,10], [5]):
    print(x)

In [ ]:
for x in get_performance_decrease(70, mnist_data, [5,7,10], [5]):
    print(x)

### Performance Analysis as the number of rounds increases

In [ ]:
# Client selection 100%

# Real-Time, single and double round
print("Mean performance decrease: Single Round, Client Selection 100%")
print("First byzantine:", mean_at_byzantine_n(1, 1, 100, mnist_data_rtime))
print("Second byzantine:", mean_at_byzantine_n(2, 1, 100, mnist_data_rtime))
print("Third byzantine:", mean_at_byzantine_n(3, 1, 100, mnist_data_rtime))
print()

print("Mean performance decrease: Double Round, Client Selection 100%")
print("First byzantine:", mean_at_byzantine_n(1, 2, 100, mnist_data_rtime))
print("Second byzantine:", mean_at_byzantine_n(2, 2, 100, mnist_data_rtime))
print("Third byzantine:", mean_at_byzantine_n(3, 2, 100, mnist_data_rtime))
print()

# Persistent ds, five rounds
print("Mean performance decrease: Five Rounds, Client Selection 100%")
print("First byzantine:", mean_at_byzantine_n(1, 5, 100, mnist_data))
print("Second byzantine:", mean_at_byzantine_n(2, 5, 100, mnist_data))
print("Third byzantine:", mean_at_byzantine_n(3, 5, 100, mnist_data))

In [ ]:
# Client selection 80%

# Real-Time, single and double round
print("Mean performance decrease: Single Round, Client Selection 80%")
print("First byzantine:", mean_at_byzantine_n(1, 1, 80, mnist_data_rtime))
print("Second byzantine:", mean_at_byzantine_n(2, 1, 80, mnist_data_rtime))
print("Third byzantine:", mean_at_byzantine_n(3, 1, 80, mnist_data_rtime))
print()

print("Mean performance decrease: Double Round, Client Selection 80%")
print("First byzantine:", mean_at_byzantine_n(1, 2, 80, mnist_data_rtime))
print("Second byzantine:", mean_at_byzantine_n(2, 2, 80, mnist_data_rtime))
print("Third byzantine:", mean_at_byzantine_n(3, 2, 80, mnist_data_rtime))
print()

# Persistent ds, five rounds
print("Mean performance decrease: Five Rounds, Client Selection 80%")
print("First byzantine:", mean_at_byzantine_n(1, 5, 80, mnist_data))
print("Second byzantine:", mean_at_byzantine_n(2, 5, 80, mnist_data))
print("Third byzantine:", mean_at_byzantine_n(3, 5, 80, mnist_data))

In [ ]:
# Client selection 70%

# Real-Time, single and double round
print("Mean performance decrease: Single Round, Client Selection 70%")
print("First byzantine:", mean_at_byzantine_n(1, 1, 70, mnist_data_rtime))
print("Second byzantine:", mean_at_byzantine_n(2, 1, 70, mnist_data_rtime))
print("Third byzantine:", mean_at_byzantine_n(3, 1, 70, mnist_data_rtime))
print()

print("Mean performance decrease: Double Round, Client Selection 70%")
print("First byzantine:", mean_at_byzantine_n(1, 2, 70, mnist_data_rtime))
print("Second byzantine:", mean_at_byzantine_n(2, 2, 70, mnist_data_rtime))
print("Third byzantine:", mean_at_byzantine_n(3, 2, 70, mnist_data_rtime))
print()

# Persistent ds, five rounds
print("Mean performance decrease: Five Rounds, Client Selection 70%")
print("First byzantine:", mean_at_byzantine_n(1, 5, 70, mnist_data))
print("Second byzantine:", mean_at_byzantine_n(2, 5, 70, mnist_data))
print("Third byzantine:", mean_at_byzantine_n(3, 5, 70, mnist_data))

## MNIST Analysis

### Performance analysis as the total number of nodes increases

#### Real-Time Task

In [ ]:
for x in get_performance_decrease(100, fashionmnist_data_rtime, [4,5,6], [1, 2]):
    print(x)


4 clients, 1 rounds: 0.9027 -> 0.8982 -> 0.7323 -> 0.711
L'abbattimento maggiore delle performance si ha introducento il secondo nodo bizantino

4 clients, 2 rounds: 0.9012 -> 0.9023 -> 0.9028 -> 0.9027
Abbattimento praticamente nullo

5 clients, 1 rounds: 0.9005 -> 0.8987 -> 0.8057 -> 0.7099
Abbattimento delle performance minore rispetto al caso precedente ma comunque significativo: 9% all'introduzione del secondo nodo bizantino, 10% al terzo.

5 clients, 2 rounds: 0.8978 -> 0.901 -> 0.899 -> 0.8981
Abbattimento praticamente nullo 

6 clients, 1 rounds: 0.8976 -> 0.8949 -> 0.8666 -> 0.7117
Prestazioni leggermente più basse inizialmente, ma più robusto. Solo al terzo nodo bizantino si ha un abbattimento significativo delle performance.

6 clients, 2 rounds: 0.894 -> 0.8958 -> 0.8923 -> 0.8922
Abbattimento praticamente nullo

L'aumento del numero di round non sembra influenzare le performance, anzi, sembra migliorare la robustezza del sistema.

In [ ]:
for x in get_performance_decrease(80, fashionmnist_data_rtime, [4,5,6], [1, 2]):
    print(x)

4 client, 1 round -> 0.9006, 0.86, 0.7121, 0.7096   
L'abbattimento maggiore delle performance si ha introducento il secondo nodo bizantino

5 client, 1 round -> 0.8987, 0.8844, 0.7584, 0.7113 
L'abbattimento maggiore delle performance si ha introducento il secondo nodo bizantino

6 client, 1 round -> 0.8968, 0.8793, 0.879, 0.7222 
Prestazioni leggermente più basse inizialmente, ma più robusto. Solo al terzo nodo bizantino si ha un abbattimento significativo delle performance.

4 client, 2 round -> 0.8997, 0.8966, 0.8963, 0.8974 
Abbattimento praticamente nullo

5 client, 2 round -> 0.8974, 0.8982, 0.9002, 0.8996 
Abbattimento praticamente nullo

6 client, 2 round -> 0.8922, 0.8932, 0.8953, 0.8909 
Abbattimento praticamente nullo

In [ ]:
for x in get_performance_decrease(70, fashionmnist_data_rtime, [4,5,6], [1, 2]):
    print(x)

4 client, 1 round -> 0.8908, 0.7664, 0.7071, 0.709 
L'abbattimento maggiore delle performance si ha introducento il secondo nodo bizantino

5 client, 1 round -> 0.8968, 0.8978, 0.8621, 0.8298 
L'abbattimento maggiore delle performance si ha introducento il secondo nodo bizantino

6 client, 1 round -> 0.8953, 0.879, 0.8913, 0.7069 
Prestazioni leggermente più basse inizialmente, ma più robusto. Solo al terzo nodo bizantino si ha un abbattimento significativo delle performance.

4 client, 2 round -> 0.896, 0.8975, 0.8982, 0.8948 
Abbattimento praticamente nullo

5 client, 2 round -> 0.8938, 0.8972, 0.8962, 0.8957 
Abbattimento praticamente nullo

6 client, 2 round -> 0.8938, 0.8981, 0.894, 0.8881  
Abbattimento praticamente nullo

### Performance Analysis as the number of rounds increases

In [ ]:
print("Mean performance decrease: Single Round, Client Selection 100%")
print("First byzantine:", mean_at_byzantine_n(1, 1, 100, fashionmnist_data_rtime))
print("Second byzantine:", mean_at_byzantine_n(2, 1, 100, fashionmnist_data_rtime))
print("Third byzantine:", mean_at_byzantine_n(3, 1, 100, fashionmnist_data_rtime))

print("Mean performance decrease: Double Round, Client Selection 100%")
print("First byzantine:", mean_at_byzantine_n(1, 2, 100, fashionmnist_data_rtime))
print("Second byzantine:", mean_at_byzantine_n(2, 2, 100, fashionmnist_data_rtime))
print("Third byzantine:", mean_at_byzantine_n(3, 2, 100, fashionmnist_data_rtime))

In [ ]:
print("Mean performance decrease: Single Round, Client Selection 80%")
print("First byzantine:", mean_at_byzantine_n(1, 1, 80, fashionmnist_data_rtime))
print("Second byzantine:", mean_at_byzantine_n(2, 1, 80, fashionmnist_data_rtime))
print("Third byzantine:", mean_at_byzantine_n(3, 1, 80, fashionmnist_data_rtime))

print("Mean performance decrease: Double Round, Client Selection 80%")
print("First byzantine:", mean_at_byzantine_n(1, 2, 80, fashionmnist_data_rtime))
print("Second byzantine:", mean_at_byzantine_n(2, 2, 80, fashionmnist_data_rtime))
print("Third byzantine:", mean_at_byzantine_n(3, 2, 80, fashionmnist_data_rtime))

In [ ]:
print("Mean performance decrease: Single Round, Client Selection 70%")
print("First byzantine:", mean_at_byzantine_n(1, 1, 70, fashionmnist_data_rtime))
print("Second byzantine:", mean_at_byzantine_n(2, 1, 70, fashionmnist_data_rtime))
print("Third byzantine:", mean_at_byzantine_n(3, 1, 70, fashionmnist_data_rtime))

print("Mean performance decrease: Double Round, Client Selection 70%")
print("First byzantine:", mean_at_byzantine_n(1, 2, 70, fashionmnist_data_rtime))
print("Second byzantine:", mean_at_byzantine_n(2, 2, 70, fashionmnist_data_rtime))
print("Third byzantine:", mean_at_byzantine_n(3, 2, 70, fashionmnist_data_rtime))

In [ ]:
for x in get_performance_decrease(100, fashionmnist_data, [5, 7, 10], [5]):
    print(x)

5 clients, 5 rounds: 0.914 -> 0.9104 -> 0.8976 -> 0.7859
Abbattimento nullo con 1 bizantino, basso al secondo, alto al terzo.

7 clients, 5 rounds: 0.9117 -> 0.9108 -> 0.9042 -> 0.8955
Abbattimento nullo con 1 bizantino, basso al secondo e al terzo.

10 clients, 5 rounds: 0.9101 -> 0.9115 -> 0.9062 -> 0.9063
Abbattimento nullo con 1 bizantino, basso al secondo e di nuovo nullo al terzo.

In [ ]:
for x in get_performance_decrease(80, fashionmnist_data, [5, 7, 10], [5]):
    print(x)

5 clients, 5 rounds: 0.913 -> 0.9092 -> 0.908 -> 0.8727
Abbattimento delle prestazioni basso ad ogni nodo introdotto.

7 clients, 5 rounds: 0.91 -> 0.9093 -> 0.9039 -> 0.9018
Abbattimento delle prestazioni quasi nullo

10 clients, 5 rounds: 0.9067 -> 0.9072 -> 0.9045 -> 0.8932
Abbattimneto delle prestazioni quasi nullo. ~1% con 3 nodi bizantini

In [ ]:
for x in get_performance_decrease(70, fashionmnist_data, [5, 7, 10], [5]):
    print(x)

5 clients, 5 rounds: 0.9132 -> 0.9084 -> 0.9096 -> 0.9011
Abbattimento quasi nullo

7 clients, 5 rounds: 0.9118 -> 0.9058 -> 0.8959 -> 0.9049
Abbattimento quasi nullo

10 clients, 5 rounds: 0.9078 -> 0.9094 -> 0.9028 -> 0.8986
Abbattimento quasi nullo